<a href="https://colab.research.google.com/github/creepereye1204/Crime-prevention-project/blob/renewal/FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# dataset/
#     anchor/
#         anchor1.jpg
#         anchor2.jpg
#         ...
#     positive/
#         positive1.jpg
#         positive2.jpg
#         ...
#     negative/
#         negative1.jpg
#         negative2.jpg
#         ...


In [ ]:
#CPU로

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image

# 1. 모델 정의
model = InceptionResnetV1(pretrained='vggface2').train()

# 2. 트립렛 손실 함수 정의
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        pos_distance = (anchor - positive).pow(2).sum(1)  # 유클리드 거리의 제곱
        neg_distance = (anchor - negative).pow(2).sum(1)  # 유클리드 거리의 제곱
        loss = torch.relu(pos_distance - neg_distance + self.margin)
        return loss.mean()

criterion = TripletLoss(margin=1.0)

# 3. 옵티마이저 설정
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 데이터셋 클래스 정의
class TripletDataset(Dataset):
    def __init__(self, anchor_dir, positive_dir, negative_dir, transform=None):
        self.anchor_dir = anchor_dir
        self.positive_dir = positive_dir
        self.negative_dir = negative_dir
        self.transform = transform
        self.anchor_images = os.listdir(anchor_dir)
        self.positive_images = os.listdir(positive_dir)
        self.negative_images = os.listdir(negative_dir)

    def __len__(self):
        return min(len(self.anchor_images), len(self.positive_images), len(self.negative_images))

    def __getitem__(self, idx):
        anchor_path = os.path.join(self.anchor_dir, self.anchor_images[idx])
        positive_path = os.path.join(self.positive_dir, self.positive_images[idx])
        negative_path = os.path.join(self.negative_dir, self.negative_images[idx])

        anchor_image = Image.open(anchor_path).convert('RGB')
        positive_image = Image.open(positive_path).convert('RGB')
        negative_image = Image.open(negative_path).convert('RGB')

        if self.transform:
            anchor_image = self.transform(anchor_image)
            positive_image = self.transform(positive_image)
            negative_image = self.transform(negative_image)

        return anchor_image, positive_image, negative_image

# 이미지 전처리 변환 정의
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 폴더 경로 설정 (실제 경로로 대체 필요)
anchor_dir = 'path/to/dataset/anchor'
positive_dir = 'path/to/dataset/positive'
negative_dir = 'path/to/dataset/negative'

# 데이터 로더 정의
dataset = TripletDataset(anchor_dir, positive_dir, negative_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        anchor, positive, negative = data
        optimizer.zero_grad()
        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)
        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 10 == 9:  # 매 10 미니 배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')

# 4. 임베딩 벡터 계산 및 분류 함수 정의
def get_embedding(model, image):
    model.eval()
    with torch.no_grad():
        embedding = model(image.unsqueeze(0))
    return embedding

# 예시: 앵커, 양성, 음성 이미지의 임베딩 벡터 계산
anchor_image = transform(Image.open('path/to/anchor_image.jpg').convert('RGB'))
positive_image = transform(Image.open('path/to/positive_image.jpg').convert('RGB'))
negative_image = transform(Image.open('path/to/negative_image.jpg').convert('RGB'))

anchor_embedding = get_embedding(model, anchor_image)
positive_embedding = get_embedding(model, positive_image)
negative_embedding = get_embedding(model, negative_image)

# 유클리드 거리 계산
pos_distance = torch.dist(anchor_embedding, positive_embedding, p=2).item()
neg_distance = torch.dist(anchor_embedding, negative_embedding, p=2).item()

# 임계값을 기준으로 분류
threshold = 0.5  # 임계값 설정
if pos_distance < threshold:
    print("Positive (같은 클래스)")
else:
    print("Negative (다른 클래스)")

print(f'Positive Distance: {pos_distance}')
print(f'Negative Distance: {neg_distance}')


In [ ]:
# TPU로

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.data_parallel as dp
import torch_xla.utils.serialization as xser

# 1. TPU 장치 설정
device = xm.xla_device()

# 2. 모델 정의
model = InceptionResnetV1(pretrained='vggface2').train().to(device)

# 3. 트립렛 손실 함수 정의
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        pos_distance = (anchor - positive).pow(2).sum(1)  # 유클리드 거리의 제곱
        neg_distance = (anchor - negative).pow(2).sum(1)  # 유클리드 거리의 제곱
        loss = torch.relu(pos_distance - neg_distance + self.margin)
        return loss.mean()

criterion = TripletLoss(margin=1.0)

# 4. 옵티마이저 설정
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 데이터셋 클래스 정의
class TripletDataset(Dataset):
    def __init__(self, anchor_dir, positive_dir, negative_dir, transform=None):
        self.anchor_dir = anchor_dir
        self.positive_dir = positive_dir
        self.negative_dir = negative_dir
        self.transform = transform
        self.anchor_images = os.listdir(anchor_dir)
        self.positive_images = os.listdir(positive_dir)
        self.negative_images = os.listdir(negative_dir)

    def __len__(self):
        return min(len(self.anchor_images), len(self.positive_images), len(self.negative_images))

    def __getitem__(self, idx):
        anchor_path = os.path.join(self.anchor_dir, self.anchor_images[idx])
        positive_path = os.path.join(self.positive_dir, self.positive_images[idx])
        negative_path = os.path.join(self.negative_dir, self.negative_images[idx])

        anchor_image = Image.open(anchor_path).convert('RGB')
        positive_image = Image.open(positive_path).convert('RGB')
        negative_image = Image.open(negative_path).convert('RGB')

        if self.transform:
            anchor_image = self.transform(anchor_image)
            positive_image = self.transform(positive_image)
            negative_image = self.transform(negative_image)

        return anchor_image, positive_image, negative_image

# 이미지 전처리 변환 정의
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 폴더 경로 설정 (실제 경로로 대체 필요)
anchor_dir = 'path/to/dataset/anchor'
positive_dir = 'path/to/dataset/positive'
negative_dir = 'path/to/dataset/negative'

# 데이터 로더 정의
dataset = TripletDataset(anchor_dir, positive_dir, negative_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        anchor, positive, negative = data
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        optimizer.zero_grad()
        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)
        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
        loss.backward()
        xm.optimizer_step(optimizer)
        running_loss += loss.item()

        if i % 10 == 9:  # 매 10 미니 배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')

# 4. 임베딩 벡터 계산 및 분류 함수 정의
def get_embedding(model, image):
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        embedding = model(image.unsqueeze(0))
    return embedding

# 예시: 앵커, 양성, 음성 이미지의 임베딩 벡터 계산
anchor_image = transform(Image.open('path/to/anchor_image.jpg').convert('RGB'))
positive_image = transform(Image.open('path/to/positive_image.jpg').convert('RGB'))
negative_image = transform(Image.open('path/to/negative_image.jpg').convert('RGB'))

anchor_embedding = get_embedding(model, anchor_image)
positive_embedding = get_embedding(model, positive_image)
negative_embedding = get_embedding(model, negative_image)

# 유클리드 거리 계산
pos_distance = torch.dist(anchor_embedding, positive_embedding, p=2).item()
neg_distance = torch.dist(anchor_embedding, negative_embedding, p=2).item()

# 임계값을 기준으로 분류
threshold = 0.5  # 임계값 설정
if pos_distance < threshold:
    print("Positive (같은 클래스)")
else:
    print("Negative (다른 클래스)")

print(f'Positive Distance: {pos_distance}')
print(f'Negative Distance: {neg_distance}')


In [ ]:
# GPU로

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image

# 1. GPU 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 모델 정의
model = InceptionResnetV1(pretrained='vggface2').train().to(device)

# 3. 트립렛 손실 함수 정의
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        pos_distance = (anchor - positive).pow(2).sum(1)  # 유클리드 거리의 제곱
        neg_distance = (anchor - negative).pow(2).sum(1)  # 유클리드 거리의 제곱
        loss = torch.relu(pos_distance - neg_distance + self.margin)
        return loss.mean()

criterion = TripletLoss(margin=1.0)

# 4. 옵티마이저 설정
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 데이터셋 클래스 정의
class TripletDataset(Dataset):
    def __init__(self, anchor_dir, positive_dir, negative_dir, transform=None):
        self.anchor_dir = anchor_dir
        self.positive_dir = positive_dir
        self.negative_dir = negative_dir
        self.transform = transform
        self.anchor_images = os.listdir(anchor_dir)
        self.positive_images = os.listdir(positive_dir)
        self.negative_images = os.listdir(negative_dir)

    def __len__(self):
        return min(len(self.anchor_images), len(self.positive_images), len(self.negative_images))

    def __getitem__(self, idx):
        anchor_path = os.path.join(self.anchor_dir, self.anchor_images[idx])
        positive_path = os.path.join(self.positive_dir, self.positive_images[idx])
        negative_path = os.path.join(self.negative_dir, self.negative_images[idx])

        anchor_image = Image.open(anchor_path).convert('RGB')
        positive_image = Image.open(positive_path).convert('RGB')
        negative_image = Image.open(negative_path).convert('RGB')

        if self.transform:
            anchor_image = self.transform(anchor_image)
            positive_image = self.transform(positive_image)
            negative_image = self.transform(negative_image)

        return anchor_image, positive_image, negative_image

# 이미지 전처리 변환 정의
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 폴더 경로 설정 (실제 경로로 대체 필요)
anchor_dir = 'path/to/dataset/anchor'
positive_dir = 'path/to/dataset/positive'
negative_dir = 'path/to/dataset/negative'

# 데이터 로더 정의
dataset = TripletDataset(anchor_dir, positive_dir, negative_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        anchor, positive, negative = data
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        optimizer.zero_grad()
        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)
        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 10 == 9:  # 매 10 미니 배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')

# 4. 임베딩 벡터 계산 및 분류 함수 정의
def get_embedding(model, image):
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        embedding = model(image.unsqueeze(0))
    return embedding

# 예시: 앵커, 양성, 음성 이미지의 임베딩 벡터 계산
anchor_image = transform(Image.open('path/to/anchor_image.jpg').convert('RGB'))
positive_image = transform(Image.open('path/to/positive_image.jpg').convert('RGB'))
negative_image = transform(Image.open('path/to/negative_image.jpg').convert('RGB'))

anchor_embedding = get_embedding(model, anchor_image)
positive_embedding = get_embedding(model, positive_image)
negative_embedding = get_embedding(model, negative_image)

# 유클리드 거리 계산
pos_distance = torch.dist(anchor_embedding, positive_embedding, p=2).item()
neg_distance = torch.dist(anchor_embedding, negative_embedding, p=2).item()

# 임계값을 기준으로 분류
threshold = 0.5  # 임계값 설정
if pos_distance < threshold:
    print("Positive (같은 클래스)")
else:
    print("Negative (다른 클래스)")

print(f'Positive Distance: {pos_distance}')
print(f'Negative Distance: {neg_distance}')


In [ ]:
# 크롤링

In [ ]:
import os
import requests
from bs4 import BeautifulSoup

# 검색어 설정
query = '한국 연예인'  # 검색어를 변경할 수 있습니다.
url = f'https://search.naver.com/search.naver?where=image&sm=tab_jum&query={query}'

# 요청 헤더 설정 (네이버는 User-Agent 확인을 통해 봇을 차단할 수 있음)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# 이미지 URL 수집
image_urls = []
for img_tag in soup.find_all('img'):
    try:
        img_url = img_tag['data-source']
        image_urls.append(img_url)
    except KeyError:
        # data-source 속성이 없는 경우 건너뜀
        continue

# 이미지 다운로드 디렉터리 설정
os.makedirs('korean_celeb_images', exist_ok=True)

# 이미지 다운로드
for i, img_url in enumerate(image_urls):
    try:
        img_response = requests.get(img_url)
        img_response.raise_for_status()  # 요청이 성공했는지 확인
        with open(f'korean_celeb_images/korean_celeb_{i}.jpg', 'wb') as file:
            file.write(img_response.content)
        print(f'Downloaded image {i + 1}')
    except requests.exceptions.RequestException as e:
        print(f'Failed to download image {i + 1}: {e}')

print('이미지 다운로드 완료')


이미지 다운로드 완료


In [ ]:
pip install webdriver_manager

In [ ]:
import os
import random
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms




이미지 찾을 대상을 입력해주세요.백종원


AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image

# 1. GPU 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 모델 정의
model = InceptionResnetV1(pretrained='vggface2').train().to(device)

# 3. 트립렛 손실 함수 정의
class TripletDataset(Dataset):
    def __init__(self, anchor_dir, positive_dir, negative_dir, transform=None):
        self.anchor_dir = anchor_dir
        self.positive_dir = positive_dir
        self.negative_dir = negative_dir
        self.transform = transform
        self.anchor_images = sorted(os.listdir(anchor_dir))
        self.positive_images = sorted(os.listdir(positive_dir))
        self.negative_images = sorted(os.listdir(negative_dir))

        # 앵커 이미지에 해당하는 양성 이미지를 매핑합니다.
        self.anchor_to_positives = self._map_anchor_to_positives()

    def _map_anchor_to_positives(self):
        anchor_to_positives = {}
        for positive_image in self.positive_images:
            anchor_name = positive_image.split('_')[0]
            if anchor_name not in anchor_to_positives:
                anchor_to_positives[anchor_name] = []
            anchor_to_positives[anchor_name].append(positive_image)
        return anchor_to_positives

    def __len__(self):
        return len(self.anchor_images)

    def __getitem__(self, idx):
        anchor_image_name = self.anchor_images[idx]
        anchor_path = os.path.join(self.anchor_dir, anchor_image_name)

        # 앵커 이미지에 대응하는 양성 이미지 중 하나를 랜덤하게 선택합니다.
        positive_image_name = random.choice(self.anchor_to_positives[anchor_image_name.split('.')[0]])
        positive_path = os.path.join(self.positive_dir, positive_image_name)

        # 음성 이미지는 무작위로 선택합니다.
        negative_image_name = random.choice(self.negative_images)
        negative_path = os.path.join(self.negative_dir, negative_image_name)

        anchor_image = Image.open(anchor_path).convert('RGB')
        positive_image = Image.open(positive_path).convert('RGB')
        negative_image = Image.open(negative_path).convert('RGB')

        if self.transform:
            anchor_image = self.transform(anchor_image)
            positive_image = self.transform(positive_image)
            negative_image = self.transform(negative_image)

        return anchor_image, positive_image, negative_image

# 이미지 전처리 변환 정의
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 폴더 경로 설정 (실제 경로로 대체 필요)
anchor_dir = 'path/to/dataset/anchor'
positive_dir = 'path/to/dataset/positive'
negative_dir = 'path/to/dataset/negative'

# 데이터 로더 정의
dataset = TripletDataset(anchor_dir, positive_dir, negative_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        anchor, positive, negative = data
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        optimizer.zero_grad()
        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)
        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 10 == 9:  # 매 10 미니 배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')

# 4. 임베딩 벡터 계산 및 분류 함수 정의
def get_embedding(model, image):
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        embedding = model(image.unsqueeze(0))
    return embedding

# 예시: 앵커, 양성, 음성 이미지의 임베딩 벡터 계산
anchor_image = transform(Image.open('path/to/anchor_image.jpg').convert('RGB'))
positive_image = transform(Image.open('path/to/positive_image.jpg').convert('RGB'))
negative_image = transform(Image.open('path/to/negative_image.jpg').convert('RGB'))

anchor_embedding = get_embedding(model, anchor_image)
positive_embedding = get_embedding(model, positive_image)
negative_embedding = get_embedding(model, negative_image)

# 유클리드 거리 계산
pos_distance = torch.dist(anchor_embedding, positive_embedding, p=2).item()
neg_distance = torch.dist(anchor_embedding, negative_embedding, p=2).item()

# 임계값을 기준으로 분류
threshold = 0.5  # 임계값 설정
if pos_distance < threshold:
    print("Positive (같은 클래스)")
else:
    print("Negative (다른 클래스)")

print(f'Positive Distance: {pos_distance}')
print(f'Negative Distance: {neg_distance}')


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image

# 1. GPU 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 모델 정의
model = InceptionResnetV1(pretrained='vggface2').train().to(device)

# 3. 트립렛 손실 함수 정의
class TripletDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.dataset_path = dataset_path
        self.transform = transform
        self.classes = os.listdir(dataset_path)
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.imgs = self.make_dataset()

    def make_dataset(self):
        imgs = []
        for class_name in self.classes:
            class_path = os.path.join(self.dataset_path, class_name)
            img_names = os.listdir(class_path)
            anchor_imgs = [img_name for img_name in img_names if 'anchor' in img_name]
            for anchor_img in anchor_imgs:
                positive_imgs = [img_name for img_name in img_names if img_name != anchor_img and 'positive' in img_name]
                negative_imgs = [img_name for img_name in img_names if 'negative' in img_name]
                for positive_img in positive_imgs:
                    for negative_img in negative_imgs:
                        imgs.append((os.path.join(class_path, anchor_img), os.path.join(class_path, positive_img), os.path.join(class_path, negative_img)))
        return imgs

    def __getitem__(self, index):
        anchor_path, positive_path, negative_path = self.imgs[index]
        anchor_img = Image.open(anchor_path)
        positive_img = Image.open(positive_path)
        negative_img = Image.open(negative_path)

        if self.transform is not None:
            anchor_img = self.transform(anchor_img)
            positive_img = self.transform(positive_img)
            negative_img = self.transform(negative_img)

        return anchor_img, positive_img, negative_img

    def __len__(self):
        return len(self.imgs)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = TripletDataset('dataset/', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        anchor, positive, negative = data
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        optimizer.zero_grad()
        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)
        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 10 == 9:  # 매 10 미니 배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')

# 4. 임베딩 벡터 계산 및 분류 함수 정의
def get_embedding(model, image):
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        embedding = model(image.unsqueeze(0))
    return embedding

# 예시: 앵커, 양성, 음성 이미지의 임베딩 벡터 계산
anchor_image = transform(Image.open('path/to/anchor_image.jpg').convert('RGB'))
positive_image = transform(Image.open('path/to/positive_image.jpg').convert('RGB'))
negative_image = transform(Image.open('path/to/negative_image.jpg').convert('RGB'))

anchor_embedding = get_embedding(model, anchor_image)
positive_embedding = get_embedding(model, positive_image)
negative_embedding = get_embedding(model, negative_image)

# 유클리드 거리 계산
pos_distance = torch.dist(anchor_embedding, positive_embedding, p=2).item()
neg_distance = torch.dist(anchor_embedding, negative_embedding, p=2).item()

# 임계값을 기준으로 분류
threshold = 0.5  # 임계값 설정
if pos_distance < threshold:
    print("Positive (같은 클래스)")
else:
    print("Negative (다른 클래스)")

print(f'Positive Distance: {pos_distance}')
print(f'Negative Distance: {neg_distance}')


In [ ]:
# 평가(evaluation) 함수 정의
def evaluate(model, dataloader, device):
    model.eval()  # 모델을 평가 모드로 설정
    running_loss = 0.0
    with torch.no_grad():  # 기울기 계산을 비활성화하여 메모리 사용량 줄이고 계산 속도 향상
        for data in dataloader:
            anchor, positive, negative = data
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

            anchor_embedding = model(anchor)
            positive_embedding = model(positive)
            negative_embedding = model(negative)
            loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
            running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    print(f'Evaluation Loss: {avg_loss:.3f}')
    return avg_loss

# 테스트 데이터셋과 데이터로더 준비
# 테스트 데이터셋의 구조는 학습 및 평가 데이터셋과 동일하다고 가정
test_dataset = TripletDataset('test_dataset/', transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image

# 1. GPU 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 모델 정의
model = InceptionResnetV1(pretrained='vggface2').train().to(device)

# 3. 트리플렛 손실 함수와 최적화 함수 정의
criterion = nn.TripletMarginLoss(margin=1.0, p=2)
optimizer = optim.Adam(model.parameters(), lr=0.001)

class TripletDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.dataset_path = dataset_path
        self.transform = transform
        self.classes = os.listdir(dataset_path)
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.imgs = self.make_dataset()

    def make_dataset(self):
        imgs = []
        for class_name in self.classes:
            class_path = os.path.join(self.dataset_path, class_name)
            img_names = os.listdir(class_path)
            anchor_imgs = [img_name for img_name in img_names if 'anchor' in img_name]
            for anchor_img in anchor_imgs:
                positive_imgs = [img_name for img_name in img_names if img_name != anchor_img and 'positive' in img_name]
                negative_imgs = [img_name for img_name in img_names if 'negative' in img_name]
                for positive_img in positive_imgs:
                    for negative_img in negative_imgs:
                        imgs.append((os.path.join(class_path, anchor_img), os.path.join(class_path, positive_img), os.path.join(class_path, negative_img)))
        return imgs

    def __getitem__(self, index):
        anchor_path, positive_path, negative_path = self.imgs[index]
        anchor_img = Image.open(anchor_path)
        positive_img = Image.open(positive_path)
        negative_img = Image.open(negative_path)

        if self.transform is not None:
            anchor_img = self.transform(anchor_img)
            positive_img = self.transform(positive_img)
            negative_img = self.transform(negative_img)

        return anchor_img, positive_img, negative_img

    def __len__(self):
        return len(self.imgs)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = TripletDataset('dataset/', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        anchor, positive, negative = data
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        optimizer.zero_grad()
        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)
        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 10 == 9:  # 매 10 미니 배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')

# 4. 임베딩 벡터 계산 및 분류 함수 정의
def get_embedding(model, image):
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        embedding = model(image.unsqueeze(0))
    return embedding

# 예시: 앵커, 양성, 음성 이미지의 임베딩 벡터 계산
anchor_image = transform(Image.open('path/to/anchor_image.jpg').convert('RGB'))
positive_image = transform(Image.open('path/to/positive_image.jpg').convert('RGB'))
negative_image = transform(Image.open('path/to/negative_image.jpg').convert('RGB'))

anchor_embedding = get_embedding(model, anchor_image)
positive_embedding = get_embedding(model, positive_image)
negative_embedding = get_embedding(model, negative_image)

# 유클리드 거리 계산
pos_distance = torch.dist(anchor_embedding, positive_embedding, p=2).item()
neg_distance = torch.dist(anchor_embedding, negative_embedding, p=2).item()

# 임계값을 기준으로 분류
threshold = 0.5  # 임계값 설정
if pos_distance < threshold:
    print("Positive (같은 클래스)")
else:
    print("Negative (다른 클래스)")

print(f'Positive Distance: {pos_distance}')
print(f'Negative Distance: {neg_distance}')


In [ ]:
dataset/
│
├── Person1/
│   ├── anchor_001.jpg  # 앵커 이미지
│   ├── positive_001.jpg  # 긍정 이미지 (같은 클래스)
│   ├── positive_002.jpg  # 긍정 이미지 (같은 클래스)
│   ├── negative_001.jpg  # 부정 이미지 (다른 클래스)
│   └── negative_002.jpg  # 부정 이미지 (다른 클래스)
│
├── Person2/
│   ├── anchor_001.jpg  # 앵커 이미지
│   ├── positive_001.jpg  # 긍정 이미지 (같은 클래스)
│   ├── positive_002.jpg  # 긍정 이미지 (같은 클래스)
│   ├── negative_001.jpg  # 부정 이미지 (다른 클래스)
│   └── negative_002.jpg  # 부정 이미지 (다른 클래스)
│
├── Person3/
│   ├── anchor_001.jpg  # 앵커 이미지
│   ├── positive_001.jpg  # 긍정 이미지 (같은 클래스)
│   ├── positive_002.jpg  # 긍정 이미지 (같은 클래스)
│   ├── negative_001.jpg  # 부정 이미지 (다른 클래스)
│   └── negative_002.jpg  # 부정 이미지 (다른 클래스)
│
└── ...


In [ ]:
# 네, 제공하신 구조에 따라 이미지를 저장하는 방법은 트리플렛 네트워크 학습에 적합합니다. 각 '앵커' 폴더에는 해당 앵커 이미지와 그에 대응하는 긍정(positive) 및 부정(negative) 이미지가 포함되어 있습니다. 이 구조는 앵커 이미지와 긍정 이미지 간의 유사성을 학습하고, 앵커 이미지와 부정 이미지 간의 차이를 학습하는 데 도움이 됩니다.

# 그러나, 실제 코드에서 이미지 경로를 만들 때 'anchor', 'positive', 'negative'라는 단어를 포함하는 방식에 따라 이미지를 분류하므로, 폴더 구조와 이미지의 이름을 명확하게 정의해야 합니다. 제공하신 예시에서는 각 앵커 폴더에 하나의 'anchor.jpg' 이미지, 여러 개의 'positive_xx.jpg', 그리고 여러 개의 'negative_xx.jpg' 이미지가 포함되어 있습니다. 이 구조는 코드에서 정의한 `TripletDataset` 클래스의 `make_dataset` 메소드 로직과 일치합니다.

# `TripletDataset` 클래스의 `make_dataset` 메소드는 각 앵커 이미지에 대해 가능한 모든 긍정 및 부정 이미지 조합을 찾아 이미지의 트리플렛(앵커, 긍정, 부정)을 생성합니다. 이렇게 하여 학습 데이터셋에 다양성을 부여하고, 모델이 더 강력한 특징을 학습할 수 있도록 합니다.

# 따라서, 제공하신 파일 구조는 코드의 요구사항을 충족하며, 트리플렛 네트워크 학습에 적합한 방식입니다. 하지만, 실제 프로젝트에서는 클래스별로 더 많은 이미지를 포함시키고, 다양한 앵커, 긍정, 부정 이미지 조합을 실험해 볼 필요가 있습니다. 이를 통해 모델의 일반화 능력을 향상시키고, 실제 세계의 다양한 시나리오에 대응할 수 있는 robust한 모델을 개발할 수 있습니다.

# 이런 자료를 참고했어요.
# [1] AI-Hub - 한국어 이미지 설명 데이터셋 (https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=261)
# [2] 티스토리 - [PyTorch] 4-1. 나만의 이미지 데이터셋 만들기 - Real Late Starter (https://data-panic.tistory.com/13)
# [3] Superb AI - 컴퓨터 비전 데이터셋 - 공공 데이터셋 살펴보기 - 슈퍼브 블로그 (https://blog-ko.superb-ai.com/exploring-computer-vision-datasets/)
# [4] GitHub - 작은 데이터셋으로 강력한 이미지 분류 모델 설계하기 (https://keraskorea.github.io/posts/2018-10-24-little_data_powerful_model/)

# 뤼튼 사용하러 가기 > https://agent.wrtn.ai/5xb91l

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image

# 1. GPU 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 모델 정의
model = InceptionResnetV1(pretrained='vggface2').train().to(device)

# 3. 트리플렛 손실 함수와 최적화 함수 정의
criterion = nn.TripletMarginLoss(margin=1.0, p=2)
optimizer = optim.Adam(model.parameters(), lr=0.001)

class TripletDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.dataset_path = dataset_path
        self.transform = transform
        self.classes = os.listdir(dataset_path)
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.imgs = self.make_dataset()

    def make_dataset(self):
        imgs = []
        for class_name in self.classes:
            class_path = os.path.join(self.dataset_path, class_name)
            img_names = os.listdir(class_path)
            anchor_imgs = [img_name for img_name in img_names if 'anchor' in img_name]
            for anchor_img in anchor_imgs:
                positive_imgs = [img_name for img_name in img_names if img_name != anchor_img and 'positive' in img_name]
                negative_imgs = [img_name for img_name in img_names if 'negative' in img_name]
                for positive_img in positive_imgs:
                    for negative_img in negative_imgs:
                        imgs.append((os.path.join(class_path, anchor_img), os.path.join(class_path, positive_img), os.path.join(class_path, negative_img)))
        return imgs

    def __getitem__(self, index):
        anchor_path, positive_path, negative_path = self.imgs[index]
        anchor_img = Image.open(anchor_path)
        positive_img = Image.open(positive_path)
        negative_img = Image.open(negative_path)

        if self.transform is not None:
            anchor_img = self.transform(anchor_img)
            positive_img = self.transform(positive_img)
            negative_img = self.transform(negative_img)

        return anchor_img, positive_img, negative_img

    def __len__(self):
        return len(self.imgs)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = TripletDataset('dataset/', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        anchor, positive, negative = data
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        optimizer.zero_grad()
        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)
        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 10 == 9:  # 매 10 미니 배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')

# 4. 임베딩 벡터 계산 및 분류 함수 정의
def get_embedding(model, image):
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        embedding = model(image.unsqueeze(0))
    return embedding

# 예시: 앵커, 양성, 음성 이미지의 임베딩 벡터 계산
anchor_image = transform(Image.open('path/to/anchor_image.jpg').convert('RGB'))
positive_image = transform(Image.open('path/to/positive_image.jpg').convert('RGB'))
negative_image = transform(Image.open('path/to/negative_image.jpg').convert('RGB'))

anchor_embedding = get_embedding(model, anchor_image)
positive_embedding = get_embedding(model, positive_image)
negative_embedding = get_embedding(model, negative_image)

# 유클리드 거리 계산
pos_distance = torch.dist(anchor_embedding, positive_embedding, p=2).item()
neg_distance = torch.dist(anchor_embedding, negative_embedding, p=2).item()

# 임계값을 기준으로 분류
threshold = 0.5  # 임계값 설정
if pos_distance < threshold:
    print("Positive (같은 클래스)")
else:
    print("Negative (다른 클래스)")

print(f'Positive Distance: {pos_distance}')
print(f'Negative Distance: {neg_distance}')


In [ ]:
!pip install facenet_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 775.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.w

In [ ]:
######## 이코드가 진짜

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split

# 1. GPU 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 모델 정의
model = InceptionResnetV1(pretrained='vggface2').train().to(device)

# 3. 트리플렛 손실 함수와 최적화 함수 정의
criterion = nn.TripletMarginLoss(margin=1.0, p=2)
optimizer = optim.Adam(model.parameters(), lr=0.001)

class TripletDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.dataset_path = dataset_path
        self.transform = transform
        self.classes = os.listdir(dataset_path)
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.imgs = self.make_dataset()

    def make_dataset(self):
        imgs = []
        for class_name in self.classes:
            class_path = os.path.join(self.dataset_path, class_name)
            img_names = os.listdir(class_path)
            anchor_imgs = [img_name for img_name in img_names if 'anchor' in img_name]
            for anchor_img in anchor_imgs:
                positive_imgs = [img_name for img_name in img_names if img_name != anchor_img and 'positive' in img_name]
                negative_imgs = [img_name for img_name in img_names if 'negative' in img_name]
                for positive_img in positive_imgs:
                    for negative_img in negative_imgs:
                        imgs.append((os.path.join(class_path, anchor_img), os.path.join(class_path, positive_img), os.path.join(class_path, negative_img)))
        return imgs

    def __getitem__(self, index):
        anchor_path, positive_path, negative_path = self.imgs[index]
        anchor_img = Image.open(anchor_path).convert('RGB')
        positive_img = Image.open(positive_path).convert('RGB')
        negative_img = Image.open(negative_path).convert('RGB')

        if self.transform is not None:
            anchor_img = self.transform(anchor_img)
            positive_img = self.transform(positive_img)
            negative_img = self.transform(negative_img)

        return anchor_img, positive_img, negative_img

    def __len__(self):
        return len(self.imgs)

# 데이터셋 나누기
def split_dataset(dataset, train_ratio=0.7, eval_ratio=0.1, test_ratio=0.2):
    train_size = int(train_ratio * len(dataset))
    eval_size = int(eval_ratio * len(dataset))
    test_size = len(dataset) - train_size - eval_size
    return random_split(dataset, [train_size, eval_size, test_size])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = TripletDataset('dataset/', transform=transform)
train_dataset, eval_dataset, test_dataset = split_dataset(dataset)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        anchor, positive, negative = data
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        optimizer.zero_grad()
        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)
        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 10 == 9:  # 매 10 미니 배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

    # 검증 루프
    model.eval()
    eval_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(eval_loader, 0):
            anchor, positive, negative = data
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

            anchor_embedding = model(anchor)
            positive_embedding = model(positive)
            negative_embedding = model(negative)
            loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
            eval_loss += loss.item()

    print(f'[Epoch {epoch + 1}] Eval loss: {eval_loss / len(eval_loader):.3f}')

print('Finished Training')

# 테스트 루프
model.eval()
test_loss = 0.0
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        anchor, positive, negative = data
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)
        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
        test_loss += loss.item()

print(f'Test loss: {test_loss / len(test_loader):.3f}')

# 4. 임베딩 벡터 계산 및 분류 함수 정의
def get_embedding(model, image):
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        embedding = model(image.unsqueeze(0))
    return embedding

# # 예시: 앵커, 양성, 음성 이미지의 임베딩 벡터 계산
# anchor_image = transform(Image.open('path/to/anchor_image.jpg').convert('RGB'))
# positive_image = transform(Image.open('path/to/positive_image.jpg').convert('RGB'))
# negative_image = transform(Image.open('path/to/negative_image.jpg').convert('RGB'))

# anchor_embedding = get_embedding(model, anchor_image)
# positive_embedding = get_embedding(model, positive_image)
# negative_embedding = get_embedding(model, negative_image)

# # 유클리드 거리 계산
# pos_distance = torch.dist(anchor_embedding, positive_embedding, p=2).item()

# # 거리 출력
# print(f"양성 이미지와의 유클리드 거리: {pos_distance():.3f}")



  0%|          | 0.00/107M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataset.py:449: UserWarning: Length of split at index 0 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataset.py:449: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
# 진짜코드위에

In [ ]:
import torch

def expand_batch(input_tensor, target_batch_size):
    current_batch_size = input_tensor.size(0)
    if current_batch_size == 1:
        # input_tensor을 target_batch_size만큼 복사하여 새로운 배치를 만듭니다.
        expanded_tensor = input_tensor.repeat(target_batch_size, 1, 1, 1)
        return expanded_tensor
    else:
        return input_tensor

# 예제 입력 텐서
anchor = torch.randn(1, 3, 224, 224)  # 현재 배치 크기가 1인 경우



In [ ]:
# dvc데이터 버전컨트롤,
#(git, github, svn)

In [ ]:
# 버전 관리중요

In [ ]:
# 깃헙:CI,CD,협업 깃헙은 깃을 클라우드에서 쓰게만들어주는것

In [ ]:
# 모델 관리: mlflow,web tool, bentoml

In [ ]:
# 가정->임의성->데이터->설정->결과

In [ ]:
# hydra,오메가콘프

In [ ]:
# mamba,conda,poetry

In [ ]:
# 재현가능성,

In [ ]:
# 컨테이너 도커

In [ ]:
# git , dvc , docker

In [ ]:
# 깃,깃헙,도커 는 필수
#자동환경(깃헙 액션)

In [ ]:
# torch complie

In [ ]:
# ONNX 로 포멧변경 을하여 임베딩즉 디바이스에 적용

In [ ]:
# 양자화, 가지치기, 증류

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image

# 1. GPU 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 모델 정의
model = InceptionResnetV1(pretrained='vggface2').train().to(device)

# 3. 트리플렛 손실 함수와 최적화 함수 정의
criterion = nn.TripletMarginLoss(margin=1.0, p=2)
optimizer = optim.Adam(model.parameters(), lr=0.001)

class TripletDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.dataset_path = dataset_path
        self.transform = transform
        self.classes = os.listdir(dataset_path)
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.imgs = self.make_dataset()

    def make_dataset(self):
        imgs = []
        for class_name in self.classes:
            class_path = os.path.join(self.dataset_path, class_name)
            img_names = os.listdir(class_path)
            anchor_imgs = [img_name for img_name in img_names if 'anchor' in img_name]
            for anchor_img in anchor_imgs:
                positive_imgs = [img_name for img_name in img_names if img_name != anchor_img and 'positive' in img_name]
                negative_imgs = [img_name for img_name in img_names if 'negative' in img_name]
                for positive_img in positive_imgs:
                    for negative_img in negative_imgs:
                        imgs.append((os.path.join(class_path, anchor_img), os.path.join(class_path, positive_img), os.path.join(class_path, negative_img)))
        return imgs

    def __getitem__(self, index):
        anchor_path, positive_path, negative_path = self.imgs[index]
        anchor_img = Image.open(anchor_path)
        positive_img = Image.open(positive_path)
        negative_img = Image.open(negative_path)

        if self.transform is not None:
            anchor_img = self.transform(anchor_img)
            positive_img = self.transform(positive_img)
            negative_img = self.transform(negative_img)

        return anchor_img, positive_img, negative_img

    def __len__(self):
        return len(self.imgs)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = TripletDataset('dataset/', transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, drop_last=True)  # drop_last=True 추가

# 학습 루프
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        anchor, positive, negative = data
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        optimizer.zero_grad()
        target_batch_size = 32  # 원하는 배치 크기

        # 가짜 배치 크기로 확장
        anchor = expand_batch(anchor, target_batch_size)
        positive = expand_batch(positive, target_batch_size)
        negative = expand_batch(negative, target_batch_size)

        # 모델에 입력
        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)
        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()


        print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
        running_loss = 0.0

print('Finished Training')

# 4. 임베딩 벡터 계산 및 분류 함수 정의
def get_embedding(model, image):
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        embedding = model(image.unsqueeze(0))
    return embedding


[Epoch 1, Batch 1] loss: 0.103
[Epoch 2, Batch 1] loss: 0.099
[Epoch 3, Batch 1] loss: 0.100
[Epoch 4, Batch 1] loss: 0.099
[Epoch 5, Batch 1] loss: 0.103
[Epoch 6, Batch 1] loss: 0.099
[Epoch 7, Batch 1] loss: 0.106
[Epoch 8, Batch 1] loss: 0.098
[Epoch 9, Batch 1] loss: 0.102
[Epoch 10, Batch 1] loss: 0.095
[Epoch 11, Batch 1] loss: 0.099
[Epoch 12, Batch 1] loss: 0.100
[Epoch 13, Batch 1] loss: 0.100
[Epoch 14, Batch 1] loss: 0.098
[Epoch 15, Batch 1] loss: 0.098
[Epoch 16, Batch 1] loss: 0.098
[Epoch 17, Batch 1] loss: 0.099
[Epoch 18, Batch 1] loss: 0.096
[Epoch 19, Batch 1] loss: 0.101
[Epoch 20, Batch 1] loss: 0.102
[Epoch 21, Batch 1] loss: 0.102
[Epoch 22, Batch 1] loss: 0.095
[Epoch 23, Batch 1] loss: 0.094
[Epoch 24, Batch 1] loss: 0.104
[Epoch 25, Batch 1] loss: 0.097
[Epoch 26, Batch 1] loss: 0.106
[Epoch 27, Batch 1] loss: 0.098
[Epoch 28, Batch 1] loss: 0.108
[Epoch 29, Batch 1] loss: 0.094
[Epoch 30, Batch 1] loss: 0.097
[Epoch 31, Batch 1] loss: 0.102
[Epoch 32, Batch 

KeyboardInterrupt: 

In [ ]:
!pip install facenet_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 98.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.wh

[Epoch 1, Batch 1] loss: 0.102
[Epoch 2, Batch 1] loss: 0.100
[Epoch 3, Batch 1] loss: 0.103
[Epoch 4, Batch 1] loss: 0.099
[Epoch 5, Batch 1] loss: 0.097
[Epoch 6, Batch 1] loss: 0.100
[Epoch 7, Batch 1] loss: 0.099
[Epoch 8, Batch 1] loss: 0.099
[Epoch 9, Batch 1] loss: 0.103
[Epoch 10, Batch 1] loss: 0.102
[Epoch 11, Batch 1] loss: 0.101
[Epoch 12, Batch 1] loss: 0.099
[Epoch 13, Batch 1] loss: 0.099
[Epoch 14, Batch 1] loss: 0.101
[Epoch 15, Batch 1] loss: 0.096
[Epoch 16, Batch 1] loss: 0.098
[Epoch 17, Batch 1] loss: 0.100
[Epoch 18, Batch 1] loss: 0.099
[Epoch 19, Batch 1] loss: 0.096
[Epoch 20, Batch 1] loss: 0.098
[Epoch 21, Batch 1] loss: 0.097
[Epoch 22, Batch 1] loss: 0.098
[Epoch 23, Batch 1] loss: 0.104
[Epoch 24, Batch 1] loss: 0.098
[Epoch 25, Batch 1] loss: 0.098
[Epoch 26, Batch 1] loss: 0.100
[Epoch 27, Batch 1] loss: 0.094
[Epoch 28, Batch 1] loss: 0.097
[Epoch 29, Batch 1] loss: 0.101
[Epoch 30, Batch 1] loss: 0.098
[Epoch 31, Batch 1] loss: 0.101
[Epoch 32, Batch 

KeyboardInterrupt: 

In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image

# 1. GPU 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 모델 정의
model = InceptionResnetV1(pretrained='vggface2').train().to(device)

# 3. 트리플렛 손실 함수와 최적화 함수 정의
criterion = nn.TripletMarginLoss(margin=1.0, p=2)
optimizer = optim.Adam(model.parameters(), lr=0.001)

class TripletDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.dataset_path = dataset_path
        self.transform = transform
        self.classes = os.listdir(dataset_path)
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.imgs = self.make_dataset()

    def make_dataset(self):
        imgs = []
        for class_name in self.classes:
            class_path = os.path.join(self.dataset_path, class_name)
            img_names = os.listdir(class_path)
            for i in range(len(img_names) - 1):
                anchor_img = img_names[i]
                positive_img = img_names[i + 1]
                for other_class_name in self.classes:
                    if other_class_name != class_name:
                        other_class_path = os.path.join(self.dataset_path, other_class_name)
                        negative_img = random.choice(os.listdir(other_class_path))
                        imgs.append((os.path.join(class_path, anchor_img), os.path.join(class_path, positive_img), os.path.join(other_class_path, negative_img)))
        return imgs

    def __getitem__(self, index):
        anchor_path, positive_path, negative_path = self.imgs[index]
        anchor_img = Image.open(anchor_path)
        positive_img = Image.open(positive_path)
        negative_img = Image.open(negative_path)

        if self.transform is not None:
            anchor_img = self.transform(anchor_img)
            positive_img = self.transform(positive_img)
            negative_img = self.transform(negative_img)

        return anchor_img, positive_img, negative_img

    def __len__(self):
        return len(self.imgs)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = TripletDataset('dataset/', transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, drop_last=True)



# 학습 루프
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        anchor, positive, negative = data
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        optimizer.zero_grad()
        target_batch_size = 32  # 원하는 배치 크기

        # 가짜 배치 크기로 확장
        anchor = expand_batch(anchor, target_batch_size)
        positive = expand_batch(positive, target_batch_size)
        negative = expand_batch(negative, target_batch_size)

        # 모델에 입력
        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)
        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
        running_loss = 0.0

print('Finished Training')

# 4. 임베딩 벡터 계산 및 분류 함수 정의
def get_embedding(model, image):
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        embedding = model(image.unsqueeze(0))
    return embedding


In [ ]:
########################################

In [ ]:
import os
import random
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from facenet_pytorch import InceptionResnetV1

# GPU 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 정의
model = InceptionResnetV1(pretrained='vggface2').train().to(device)

# 트리플렛 손실 함수와 최적화 함수 정의
criterion = nn.TripletMarginLoss(margin=1.0, p=2)
optimizer = optim.Adam(model.parameters(), lr=0.001)

class TripletDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.dataset_path = dataset_path
        self.transform = transform
        self.classes = os.listdir(dataset_path)
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.imgs = self.make_dataset()

    def make_dataset(self):
        imgs = []
        for class_name in self.classes:
            class_path = os.path.join(self.dataset_path, class_name)
            img_names = os.listdir(class_path)
            for img_name in img_names:
                anchor_img = img_name
                positive_img = random.choice([img for img in img_names if img != anchor_img])
                negative_class = random.choice([cls for cls in self.classes if cls != class_name])
                negative_class_path = os.path.join(self.dataset_path, negative_class)
                negative_img = random.choice(os.listdir(negative_class_path))
                imgs.append((os.path.join(class_path, anchor_img),
                             os.path.join(class_path, positive_img),
                             os.path.join(negative_class_path, negative_img)))
        return imgs

    def __getitem__(self, index):
        anchor_path, positive_path, negative_path = self.imgs[index]
        anchor_img = Image.open(anchor_path).convert('RGB')
        positive_img = Image.open(positive_path).convert('RGB')
        negative_img = Image.open(negative_path).convert('RGB')

        if self.transform is not None:
            anchor_img = self.transform(anchor_img)
            positive_img = self.transform(positive_img)
            negative_img = self.transform(negative_img)

        return anchor_img, positive_img, negative_img

    def __len__(self):
        return len(self.imgs)

# 데이터셋 나누기
def split_dataset(dataset, train_ratio=0.7, eval_ratio=0.1, test_ratio=0.2):
    train_size = int(train_ratio * len(dataset))
    eval_size = int(eval_ratio * len(dataset))
    test_size = len(dataset) - train_size - eval_size
    return random_split(dataset, [train_size, eval_size, test_size])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = TripletDataset('dataset/', transform=transform)
train_dataset, eval_dataset, test_dataset = split_dataset(dataset)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        anchor, positive, negative = data
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        optimizer.zero_grad()
        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)
        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 10 == 9:  # 매 10 미니 배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

    # 검증 루프
    model.eval()
    eval_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(eval_loader, 0):
            anchor, positive, negative = data
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

            anchor_embedding = model(anchor)
            positive_embedding = model(positive)
            negative_embedding = model(negative)
            loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
            eval_loss += loss.item()

    print(f'[Epoch {epoch + 1}] Eval loss: {eval_loss / len(eval_loader):.3f}')
print('Finished Training')

# 테스트 루프
model.eval()
test_loss = 0.0
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        anchor, positive, negative = data
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)
        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)
        test_loss += loss.item()
print(f'Test loss: {test_loss / len(test_loader):.3f}')

# 임베딩 벡터 계산 및 분류 함수 정의
def get_embedding(model, image):
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        embedding = model(image.unsqueeze(0))
    return embedding
